In [4]:
try:
    from neuralprophet import NeuralProphet, set_log_level
    set_log_level("ERROR")
except:
    !pip install neuralprophet
    from neuralprophet import NeuralProphet, set_log_level
    set_log_level("ERROR")

try:
    import time
except:
    !pip install time
    import time

try:
    import pandas as pd
except:
    !pip install pandas
    import pandas as pd

try:
    import numpy as np
except:
    !pip install numpy
    import numpy as np
    
try:
    import warnings
    warnings.filterwarnings('ignore')
except:
    !pip install warnings
    warnings.filterwarnings('ignore')

try:
    import os
except:
    !pip install os
    import os

try:
    import json
except:
    !pip install json
    import json

### function starts here
def training(zone,temp):
    
    df = temp.copy()
    df = df[df['zone'] == zone]

    df['ds'] = pd.to_datetime(df['date'])

    # get the last date
    last_date = df['ds'].max()
    # print(last_date)

    # convert this date to epoch
    last_date_epoch = (last_date.value // 10**9) + 3600
    # print(last_date_epoch)

    # get current epoch
    import time
    current_epoch = int(time.time())
    # print(current_epoch)

    # add 7 days to current epoch
    future_epoch = current_epoch + 7 * 24 * 60 * 60
    # print(future_epoch)

    df_future = pd.DataFrame(np.arange(last_date_epoch, future_epoch, 60 * 60), columns=['timestamp'])
    df_future['ds'] = pd.to_datetime(df_future['timestamp'], unit='s')
    df_future.drop(['timestamp'], axis=1, inplace=True)
    df_future.head()
    df_future['zone'] = df['zone'].iloc[0]
    df_future['longitude'] = df['longitude'].iloc[0]
    df_future['latitude'] = df['latitude'].iloc[0]
    df_future.head()


    ### Temperature
    training_data = df[['ds']]
    training_data['y'] = df['temperature']
    training_data.reset_index(inplace=True, drop=True)

    # predicitng temperature by multiplicative model
    from neuralprophet import NeuralProphet, set_log_level, df_utils
    set_log_level("ERROR")
    m = NeuralProphet(
        changepoints_range=0.95,
        n_changepoints=24,
        trend_reg=1,
        weekly_seasonality=False,
        daily_seasonality=1,
        seasonality_mode="multiplicative",
    )
    m.set_plotting_backend("matplotlib")
    metrics = m.fit(training_data, freq="60min")
    future = m.make_future_dataframe(training_data, periods=len(df_future), n_historic_predictions=True)
    forecast = m.predict(future)
    df_future['temperature'] = forecast.tail(len(df_future))['yhat1'].values
    print('temperature done')


    ### PM2_5
    training_data = df[['ds']]
    training_data['y'] = df['pm2_5']
    training_data.reset_index(inplace=True, drop=True)
    # predicitng temperature by multiplicative model
    m = NeuralProphet(
        changepoints_range=0.95,
        n_changepoints=24,
        trend_reg=1,
        weekly_seasonality=False,
        daily_seasonality=1,
        seasonality_mode="multiplicative",
    )
    m.set_plotting_backend("matplotlib")
    metrics = m.fit(training_data, freq="60min")
    future = m.make_future_dataframe(training_data, periods=len(df_future), n_historic_predictions=True)
    forecast = m.predict(future)
    df_future['pm2_5'] = forecast.tail(len(df_future))['yhat1'].values
    print('pm2_5 done')


    ### PM10
    training_data = df[['ds']]
    training_data['y'] = df['pm10']
    training_data.reset_index(inplace=True, drop=True)

    # predicitng temperature by multiplicative model
    m = NeuralProphet(
        changepoints_range=0.95,
        n_changepoints=24,
        trend_reg=1,
        weekly_seasonality=False,
        daily_seasonality=1,
        seasonality_mode="multiplicative",
    )
    m.set_plotting_backend("matplotlib")
    metrics = m.fit(training_data, freq="60min")
    future = m.make_future_dataframe(training_data, periods=len(df_future), n_historic_predictions=True)
    forecast = m.predict(future)
    df_future['pm10'] = forecast.tail(len(df_future))['yhat1'].values
    print('pm10 done')


    df_future['smog'] = np.where((df_future['temperature'] < 25) & (df_future['pm10'] > 300), df_future[['pm10', 'pm2_5']].max(axis=1), np.nan)
    df_future['ds'] = pd.to_datetime(df_future['ds']).dt.strftime('%B %d, %Y, %I:%M %p')
    df_future.columns = ['date', 'zone', 'longitude', 'latitude', 'temperature', 'pm2_5', 'pm10', 'smog']


    # create directory if not exists
    if not os.path.exists('prediction'):
        os.makedirs('prediction')
    df_future.to_csv(f'prediction/{zone}.csv', index=False)
    
    print(f'{zone} done!!')
    print('-----------------------')

In [5]:
temp = pd.read_csv('raw_data.csv')

In [6]:
# delete the json file
if os.path.exists('checkpoints.json'):
    os.remove('checkpoints.json')
for i in range(60,70):
    training(i,temp) # training function
    # append checkpoints to a json file, if the file does not exist, create it
    if not os.path.exists('checkpoints.json'):
        with open('checkpoints.json', 'w') as f:
            json.dump([], f)
    # read the json file
    with open('checkpoints.json', 'r') as f:
        checkpoints = json.load(f)
    # append the checkpoint
    checkpoints.append(i)
    # write the json file
    with open('checkpoints.json', 'w') as f:
        json.dump(checkpoints, f)
    # write value of i to checkpoints.txt
    with open('checkpoints.txt', 'w') as f:
        f.write(str(i))

Finding best initial lr:   0%|          | 0/257 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 317it [00:00, ?it/s]

temperature done


Finding best initial lr:   0%|          | 0/257 [00:00<?, ?it/s]

...

Predicting: 317it [00:00, ?it/s]

pm2_5 done


Finding best initial lr:   0%|          | 0/257 [00:00<?, ?it/s]

Training: 0it [00:00, ?it/s]

Predicting: 317it [00:00, ?it/s]